In [1]:
import sys
from pathlib import Path

src_path = Path("..")
sys.path.append(str(src_path.resolve()))

from pydub import AudioSegment
from src.pipeline import VideoDubPipe
# from IPython.display import Audio


/home/alexbarrow/proj/video-dubbing-dls-project/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Pipeline

In [2]:
wav_path = "../data/test/test_wav.wav"
outputs = "../data/test/chunks"
config_path = "../config/config.yaml"
pipe = VideoDubPipe(config_path)

Init config...
Init ASR...
Init MT...
Init TTS...
Initialization completed.


In [3]:
chunks, durations, audio = pipe.transform(wav_path, outputs)

In [4]:
audio

In [30]:
chunks[1]

{'path': '../data/test/chunks/chunk_001.wav',
 'len': 9.3,
 'orig_seg': {'start': 9933, 'end': 19235},
 'speech_boundary': [{'start': 46112, 'end': 148832}],
 'asr_result': [[{'start': 0.0,
    'end': 1.84,
    'text': ' When I was first learning to meditate,'},
   {'start': 1.86,
    'end': 4.9,
    'text': ' the instruction was to simply pay attention to my breath,'},
   {'start': 4.92,
    'end': 7.3,
    'text': ' and when my mind wandered, to bring it back.'}]],
 'translated_text': 'Когда я впервые научилась медитировать, мне нужно было просто обратить внимание на мое дыхание, а когда мой разум блуждал, вернуть его.'}

In [ ]:
sample_rate = 16000
ps = 0.005

lengths, sp_boudary, orig_boundary = [], [], []

for chunk in chunks.values():
    lengths.append(chunk["len"])
    orig_boundary.append(
        {
            "start": round(chunk["orig_seg"]["start"] / 1000, 2),
            "end": round(chunk["orig_seg"]["end"] / 1000, 2),
        }
    )
    if not chunk["speech_boundary"]:
        sp_boudary.append([])
    else:
        orig_start = round(chunk["speech_boundary"][0]["start"] / sample_rate, 2)
        orig_end = round(chunk["speech_boundary"][0]["end"] / sample_rate, 2)
        sp_boudary.append([{"start": orig_start, "end": orig_end}])


pauses = []
for ai, bi in zip(sp_boudary, orig_boundary):
    if not ai:
        left_pause = 0.0
        right_pause = bi["end"] - bi["start"]
    else:
        left_pause = ai[0]["start"]
        right_pause = bi["end"] - bi["start"] - ai[-1]["end"]

    pauses.append([left_pause, right_pause])

for i in range(len(orig_boundary) - 1):
    gap = orig_boundary[i + 1]["start"] - orig_boundary[i]["end"]

    if gap > 0:
        shared_pause = gap / 2 - ps / 2
        shared_pause = max(shared_pause, 0.0)

        pauses[i][1] = shared_pause
        pauses[i + 1][0] = shared_pause


new_segs = []
i = 0
for length, sb, dur, p in zip(lengths, sp_boudary, durations, pauses):
    print(i)
    i += 1
    if not sb:
        new_segs.append({})
        continue

    orig_duration = sb[0]["end"] - sb[0]["start"]
    delta = dur - orig_duration

    # print(length, orig_duration, dur, delta) #######

    if delta <= 0:  # correct (речь синтетическая меньше оригинала)
        print("delta <= 0")
        mid_orig = (sb[0]["end"] + sb[0]["start"]) / 2
        synth_start = mid_orig - dur / 2
        synth_end = synth_start + dur

        if synth_start < 0:
            synth_start = 0
            synth_end = synth_start + dur
        if synth_end > length:
            synth_end = length
            synth_start = synth_end - dur
        print("sucks")
        new_segs.append({"start": synth_start, "end": synth_end})
        continue

    left_indend = sb[0]["start"] - p[0]
    right_indend = sb[0]["end"] + p[1]
    print(p)
    print("bd: ", sb[0]["start"], sb[0]["end"])
    print(left_indend, right_indend)
    new_segs.append({"start": left_indend, "end": right_indend})

0
1
[0.7875000000000001, 0.797499999999999]
bd:  2.88 9.3
2.0925 10.0975
2
[0.797499999999999, 0.5974999999999997]
bd:  0.0 0.81
-0.797499999999999 1.4074999999999998
3
[0.5974999999999997, 0.4574999999999991]
bd:  0.0 2.45
-0.5974999999999997 2.9074999999999993
4
[0.4574999999999991, 0.5575000000000006]
bd:  0.07 2.03
-0.38749999999999907 2.5875000000000004
5
[0.5575000000000006, 0.377499999999999]
bd:  0.0 2.62
-0.5575000000000006 2.997499999999999
6
delta <= 0
sucks


In [6]:
sp_boudary

[[],
 [{'start': 2.88, 'end': 9.3}],
 [{'start': 0.0, 'end': 0.81}],
 [{'start': 0.0, 'end': 2.45}],
 [{'start': 0.07, 'end': 2.03}],
 [{'start': 0.0, 'end': 2.62}],
 [{'start': 0.0, 'end': 2.49}]]

In [16]:
lengths

[8.4, 9.3, 0.8, 2.4, 2.0, 2.6, 2.8]

In [5]:
new_segs

[{},
 {'start': 2.0925, 'end': 10.0875},
 {'start': -0.797499999999999, 'end': 1.607499999999999},
 {'start': -0.5974999999999997, 'end': 3.0475},
 {'start': -0.38749999999999907, 'end': 2.487499999999999},
 {'start': -0.5575000000000006, 'end': 3.1775000000000007},
 {'start': 0.1100000000000001, 'end': 2.38}]

In [6]:
audio

In [3]:
pipe.transform(wav_path, outputs)

In [ ]:
path = "../data/test/chunks/chunk_00"
path + "0.wav"

'../data/test/chunks/chunk_000.wav'

In [ ]:
path = "../data/test/chunks/chunk_00"

ch0 = AudioSegment.from_file(path + "0.wav")
ch1 = AudioSegment.from_file(path + "1.wav")
ch2 = AudioSegment.from_file(path + "2.wav")
ch3 = AudioSegment.from_file(path + "3.wav")
ch4 = AudioSegment.from_file(path + "4.wav")
ch5 = AudioSegment.from_file(path + "5.wav")
ch6 = AudioSegment.from_file(path + "6.wav")

chunks = [ch0, ch1, ch2, ch3, ch4, ch5, ch6]

In [ ]:
chunks_len = len(ch0) + len(ch1) + len(ch2) + len(ch3) + len(ch4) + len(ch5) + len(ch6)

In [9]:
oring_audio = AudioSegment.from_file(wav_path)

In [14]:
len(oring_audio) - chunks_len

3568

In [10]:
from pydub.silence import detect_nonsilent

In [ ]:
len_orig_audio = len(oring_audio)
segments = detect_nonsilent(oring_audio, min_silence_len=700, silence_thresh=-40)

In [12]:
segments

[[0, 8351],
 [9933, 19235],
 [20827, 21633],
 [22831, 25280],
 [26199, 28230],
 [29350, 31968],
 [32725, 35500]]

In [20]:
[len(chunk) for chunk in chunks]

[8351, 9302, 806, 2449, 2031, 2618, 2775]

In [21]:
len_orig_audio

35500

In [28]:
segments[-1][1]

35500

In [ ]:
reconstructed = AudioSegment.silent(duration=0)
current_position = 0
last_chunk_end = segments[-1][1]

for (start, end), chunk in zip(segments, chunks):
    if start > current_position:
        silence_duration = start - current_position
        reconstructed += AudioSegment.silent(duration=silence_duration)

    reconstructed += chunk
    current_position = end

    if current_position == last_chunk_end:
        print(len(reconstructed))
        print("END OF END")
        reconstructed += AudioSegment.silent(
            duration=(len_orig_audio - current_position)
        )
        print(len(reconstructed))


35499
END OF END
35499


In [19]:
print(len(reconstructed))

113801
